In [1]:
import os

In [2]:
import pandas as pd
import numpy as np
import nltk

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
DATA_DIR = os.path.abspath('./datasets/output/data.csv')

In [5]:
#nltk.download('stopwords')

## Loading Data

In [6]:
df = pd.read_csv(DATA_DIR,index_col=0)
df.head()

,class,text
id,,
182,Interview,"﻿विडम्बना, ओलीको सत्ता सवारी पनि सिंहदरबारमा फ..."
127,Interview,﻿नेपाली कांग्रेसको सभापतिमा रामचन्द्र पौडेलको ...
85,Interview,﻿\nदुर्भाग्य नै भन्नुपर्छ हाम्रो प्राथमिकता पे...
162,Interview,"﻿नयनराज पाण्डे, लेखक\nविमोचन हुनै लागेको तपाईं..."
13,Interview,"﻿- भवन भट्ट, उपाध्यक्ष, गैर आवासीय नेपाली संघ\..."


In [7]:
df.dropna(inplace=True)

In [8]:
y = df['class']
X = df['text']

In [9]:
X[0]

'\ufeffविडम्बना, ओलीको सत्ता सवारी पनि सिंहदरबारमा फोटो झुण्ड्याउने मार्गतिरै अघि बढिरहेको छ।\nकल्पना गर्नुुस्, कसैले तपाईंको कन्चटमा पेस्तोल राख्दै तत्काल ‘एमाले अध्यक्ष तथा प्रधानमन्त्री केपी शर्मा ओली, एनेकपा माओवादीका अध्यक्ष पुष्पकमल दाहाल र नेपाली कांग्रेसका सभापति शेरबहादुर देउवामध्ये उत्कृष्ट को हो?’ भनी सोध्यो। के तपाईं निमेषभरमै एकजनाको नाम भन्न सक्नुहोला? जवाफ सहज छैन।\nहामीले माने पनि नमाने पनि वर्तमान नेपाली राजनीतिका त्रिमूर्ति यिनै हुन्। यिनकै विवेक, बेवकूफी, बेहोशीले नै नेपालको दशा र दिशा निर्धारित गरिरहेको छ।\nअहिले एमाले अध्यक्ष ओली सत्ता राजनीतिको केन्द्रमा छन्। पार्टी प्रमुख र प्रधानमन्त्रीमा उनलाई प्राप्त यो पहिलो मौका संभवतः अन्तिम पनि हो, जसमा उनकै स्वीकारोक्ति पटकपटक सुनिएकै छ।\nपार्टी नेतृत्वमा चुनिंदा अरूहरू जस्तो आफूसँग प्रशस्त समय नभएका कारण छिटोछिटो गर्ने, निःसन्तान रहेकाले धन सञ्चित गर्नु नपर्ने जस्ता भनाइले उनले चर्चा कमाएका थिए। पार्टी अध्यक्षका रूपमा संविधानलेखनमा देखाएको जुझारु भूमिकाले पनि नागरिकले स्वाभाविक रूपमा उनीबाट केही अपेक्षा राखेका थिए।\nतर, 

In [10]:
labels=list(set(df['class']))
labels

['World',
 'Entertainment',
 'Sports',
 'Literature',
 'Agriculture',
 'Blog',
 'Business',
 'Education',
 'Society',
 'Economy',
 'Automobiles',
 'Migration',
 'Bank',
 'Interview',
 'Opinion',
 'Technology',
 'Tourism',
 'Employment',
 'Politics']

In [11]:
len(labels)

19

### Processing Data

In [12]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
nepali_stop_words = set(stopwords.words('nepali'))

In [14]:
# tfidfVectorizer = TfidfVectorizer(stop_words=nepali_stop_words,encoding='utf-8',decode_error='ignore')

In [40]:
def split_token(x):
    return x.split(" ")
tfidfVectorizer = TfidfVectorizer(tokenizer= split_token,
                                  sublinear_tf=True, encoding='utf-8',
                                  decode_error='ignore',
                                  max_df=0.5,
                                  min_df=10,
                                  stop_words=nepali_stop_words)

In [41]:
X_vectorized = tfidfVectorizer.fit_transform(X)

/home/ash/Desktop/projects/nepali_news_classifier_recommender/venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['अक', 'अग', 'अझ', 'अन', 'अर', 'आजक', 'आत', 'आद', 'आफ', 'आय', 'ईक', 'उद', 'उनक', 'उनल', 'उह', 'एउट', 'एन', 'कत', 'कम', 'कस', 'कसर', 'कह', 'गत', 'गय', 'गर', 'चम', 'छन', 'जत', 'जबक', 'जस', 'जसक', 'जसब', 'जसम', 'जसल', 'जह', 'तत', 'तथ', 'तदन', 'तप', 'तवम', 'नज', 'नत', 'नभन', 'नय', 'पक', 'पछ', 'पन', 'पय', 'पर', 'पष', 'पह', 'बन', 'बर', 'भएक', 'भय', 'भव', 'मल', 'यत', 'यथ', 'यद', 'यप', 'यसक', 'यसपछ', 'यसब', 'यसर', 'यह', 'रण', 'रत', 'रमश', 'रह', 'लस', 'वर', 'सक', 'सट', 'सध', 'सपछ', 'सब', 'सम', 'सर', 'सह', 'हन', 'हर', 'हरण'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [42]:
X_vectorized

<5475x3170 sparse matrix of type '<class 'numpy.float64'>'
	with 498939 stored elements in Compressed Sparse Row format>

In [18]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [19]:
X_train.shape, y_train.shape

((4380,), (4380,))

In [20]:
X_test.shape, y_test.shape

((1095,), (1095,))

## SVC Classifier

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, chi2

In [31]:
svc_clf = Pipeline([
    ('vectorizer',tfidfVectorizer),
    ('chi2',SelectKBest(chi2,k=5000)),
    ('clf',SVC())
])
svc_clf_no_feat_sel = Pipeline([
    ('vectorizer',tfidfVectorizer),

    ('clf',SVC())
])

In [32]:
svc_clf.fit(X_train,y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(decode_error='ignore', max_df=0.5, min_df=10,
                                 stop_words={'अक्सर', 'अगाडी', 'अझै', 'अनुसार',
                                             'अन्तर्गत', 'अन्य', 'अन्यत्र',
                                             'अन्यथा', 'अब', 'अरु', 'अरुलाई',
                                             'अर्को', 'अर्थात', 'अर्थात्',
                                             'अलग', 'आए', 'आजको', 'आत्म', 'आदि',
                                             'आफू', 'आफूलाई', 'आफ्नै', 'आफ्नो',
                                             'आयो', 'उदाहरण', 'उनको', 'उनले',
                                             'उप', 'उहालाई', 'एउटै', ...},
                                 sublinear_tf=True,
                                 tokenizer=<function split_token at 0x7fa0670850d0>)),
                ('chi2',
                 SelectKBest(k=5000,
                             score_func=<function chi2 at 0x7fa069

In [33]:
svc_clf_no_feat_sel.fit(X_train,y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(decode_error='ignore', max_df=0.5, min_df=10,
                                 stop_words={'अक्सर', 'अगाडी', 'अझै', 'अनुसार',
                                             'अन्तर्गत', 'अन्य', 'अन्यत्र',
                                             'अन्यथा', 'अब', 'अरु', 'अरुलाई',
                                             'अर्को', 'अर्थात', 'अर्थात्',
                                             'अलग', 'आए', 'आजको', 'आत्म', 'आदि',
                                             'आफू', 'आफूलाई', 'आफ्नै', 'आफ्नो',
                                             'आयो', 'उदाहरण', 'उनको', 'उनले',
                                             'उप', 'उहालाई', 'एउटै', ...},
                                 sublinear_tf=True,
                                 tokenizer=<function split_token at 0x7fa0670850d0>)),
                ('clf', SVC())])

In [34]:
print("Accuracy on train Set : ", svc_clf.score(X_train,y_train))
print("Accuracy on test Set : ", svc_clf.score(X_test,y_test))


Accuracy on train Set :  0.9671232876712329
Accuracy on test Set :  0.7378995433789954


In [35]:
print("Accuracy on train Set : ", svc_clf_no_feat_sel.score(X_train,y_train))
print("Accuracy on test Set : ", svc_clf_no_feat_sel.score(X_test,y_test))

KeyboardInterrupt: 

In [36]:
import joblib

joblib.dump(svc_clf, 'svc_clf_pipeline.pkl')


['svc_clf_pipeline.pkl']

### Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_clf = Pipeline([
    ('vectorizer',tfidfVectorizer),
    ('chi2',SelectKBest(chi2,k=5000)),
    ('clf',RandomForestClassifier(
            n_estimators=2000, 
            max_depth=20, 
            min_samples_split=6, 
            min_samples_leaf=5, 
            bootstrap=True,
            verbose=1,
            n_jobs = -1
        )
    )
])
rf_clf_no_feat_sel = Pipeline([
    ('vectorizer',tfidfVectorizer),
    ('clf',RandomForestClassifier(
            n_estimators=2000, 
            max_depth=20, 
            min_samples_split=6, 
            min_samples_leaf=5, 
            bootstrap=True,
            verbose=1,
            n_jobs = -1
        )
    )
])

In [ ]:
rf_clf.fit(X_train,y_train)
rf_clf_no_feat_sel.fit(X_train,y_train)

In [ ]:
print("Accuracy on train Set : ", rf_clf.score(X_train,y_train))
print("Accuracy on test Set : ", rf_clf.score(X_test,y_test))

In [ ]:
print("Accuracy on train Set : ", rf_clf_no_feat_sel.score(X_train,y_train))
print("Accuracy on test Set : ", rf_clf_no_feat_sel.score(X_test,y_test))

### Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB,BernoulliNB

In [ ]:
bnb_clf = Pipeline([
    ('vect', tfidfVectorizer),
    ('clf', BernoulliNB(alpha=0.01))
])

mnb_clf = Pipeline([
    ('vect', tfidfVectorizer),
    ('clf', MultinomialNB(alpha=0.01, fit_prior=True))
])

In [ ]:
bnb_clf.fit(X_train,y_train)

In [ ]:
print("Accuracy on train Set : ", bnb_clf.score(X_train,y_train))
print("Accuracy on test Set : ", bnb_clf.score(X_test,y_test))

In [ ]:
mnb_clf.fit(X_train,y_train)

In [ ]:
print("Accuracy on train Set : ", mnb_clf.score(X_train,y_train))
print("Accuracy on test Set : ", mnb_clf.score(X_test,y_test))

## Classification Report


In [ ]:
from sklearn.metrics import classification_report

#### SVM

In [ ]:
y_pred = svc_clf.predict(X_test)
print(classification_report(y_test,y_pred))

In [ ]:
y_pred = svc_clf_no_feat_sel.predict(X_test)
print(classification_report(y_test,y_pred))

### Naive Bayes

In [ ]:
y_pred = bnb_clf.predict(X_test)
print(classification_report(y_test,y_pred))

In [ ]:
y_pred = mnb_clf.predict(X_test)
print(classification_report(y_test,y_pred))

### Random Forest

In [ ]:
y_pred = rf_clf.predict(X_test)
print(classification_report(y_test,y_pred))

In [ ]:
y_pred = rf_clf_no_feat_sel.predict(X_test)
print(classification_report(y_test,y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

In [ ]:
plot_confusion_matrix(rf_clf_no_feat_sel,y_test,y_pred)

In [ ]:
best_classifier = svc_clf

In [ ]:
best_classifier.named_steps['clf']

In [ ]:
import pickle
with open('svc_clf_test_set_72_acc.pkl','wb') as f:
    pickle.dump(best_classifier.named_steps['clf'], f) 


In [ ]:
with open('svc_clf_test_set_72_acc.pkl','rb') as f:
    model = pickle.load(f)

In [ ]:
model

In [30]:
import pickle
with open('tfid_vectorizer.pkl','wb') as f:
    pickle.dump(tfidfVectorizer, f) 

### Classification Task

In [ ]:
text = '''
९ असार, काठमाडौं । युरोकप २०२० अन्तर्गत आज समूह चरणका अन्तिम चार खेल हुँदैछ । आजको खेलहरु पनि समूह चरणका खेल सकिनेछ र अन्तिम १६ को समीकरण तय हुनेछ ।
अहिले चार समूहको विजेता र समूह विजेता अनि दुई उत्कृष्ट उपविजेताले अन्तिम १६ मा स्थान बनाएका छन् ।
यस्तै समूह इ र एफबाट पनि अन्तिम १६ मा पुग्ने एक एक टिमको टुङ्गो लागेको छ ।
आजको खेलपछि दुवै समूहको विजेता र उपविजेता अनि तेस्रो हुने मध्ये उत्कृष्ट चार टोलीको टुङ्गो लाग्नेछ ।
आज समूह इ अन्तर्गत स्लोभाकिया र स्पेन तथा पोल्याण्ड र स्वीडेनबीचको खेल राति ९:४५ बजे सुरु हुनेछ ।
स्पेनले घरेलु मैदान सेभियामा स्लोभाकियासँग खेल्नेछ । यसअघिका दुई खेल बराबरी खेलेको स्पेनलाई शीर्ष दुईमा रहन जितको आवश्यकता छ । स्लोभाकिया भने बराबरीले पनि नकआउट चरणमा पुग्न सक्नेछ ।
उता स्वीडेन र पोल्याण्ड रुसको सेन्ट पिटर्सवर्गमा भिड्नेछन् । २ खेलबाट ४ अंक जोडेको स्वीडेनले नकआउटमा स्थानमा पक्का गरिसकेको छ । पोल्याण्डमा पराजित गरेमा स्वीडेन समूह विजेता बन्नेछ।
यसैगरी राति १२:४५ बजे समूह एफको दुई महत्वपूर्ण खेल हुनेछ ।
ग्रुप अफ डेथको रुपमा रहेको यस समूहमा विश्वकप विजेतार य रोपेली विजेतासहित तीन पूर्व विजेता छन् ।
आज राति विश्व विजेता फ्रान्स र युरोपेली विजेता पोर्चुगलबीच बहुप्रतिक्षित भिडन्त हुनेछ । गत संस्करणमा फाइनल खेलेका यी दुई टोली यस पटक समूह चरणमै भिड्दैछन् ।
यस्तै जर्मनीले घरेलु मैदानमा हंगेरीसँग खेल्नेछ ।
फ्रान्स ४ अंकसहित शीर्ष स्थानमा छ र जस्तो नतिजा आए पनि नकआउटमा प्रवेश गर्नेछ । जर्मनी र पोर्चुगलको समान ३ र हंगेरीको १ अंक छ ।
पोर्चुगललाई पराजित गरेमा फ्रान्स समूह विजेता बन्नेछ । यस्तै पोर्चुगललाई पनि फ्रान्सलाई हराउँदै समूह विजेता बन्ने अवसर छ तर त्यसका लागि जर्मनी र हंगेरीबीचको खेलको नतिजा कुर्नुपर्नेछ ।
यदि फ्रान्स र हंगेरी विजयी भएमा भने पोर्चुगल अन्तिम स्थानमा हुनेछ । जर्मनीले हंगेरीलाई पराजित गरेमा र फ्रान्ससँग एक गोल अन्तरले हारेमा पनि पोर्चुगल उत्कृष्ट तेस्रो टिममध्ये एक हुँदै नकआउट चरणमा प्रवेश गर्नेछ।
'''
label = 'sports'

In [ ]:
svc_clf.predict([text])

In [ ]:
mnb_clf.predict([text])

In [ ]:
bnb_clf.predict([text])

In [ ]:
rf_clf.predict([text])

In [37]:
text = '''
९ असार, काठमाडौं । नेपाल स्टक एक्सचेञ्ज (नेप्से) को दुई वटा वेवसाइटहरु सञ्चालनमा छन् । दुबै वेवसाइट (नयाँ र पुरानो) मा सेयर बजार प्रत्यक्ष हेर्न सकिन्छ । दुई वेवसाइटमा देखिने तथ्यांक एउटै हुन् ।
तर बुधबारको कारोबारमा भने फरक देखियो । नेप्सेकेा पुरानो वेवसाइटमा सेयर कारोबार मापक नेप्से परिसूचक ५१.६५ अंकले बजार घट्यो । बधबार बजार २ हजार ८ सय ६२ दशमलव १८ विन्दुमा पुगेर बन्द भएको देखिन्छ । उता नयाँ वेवसाइटमा भने ५३.८८ अंकले घटेर २ हजार ८ सय ५९ दशमलव ९६ अंकमा झरेको छ 
यी दुई वेवसाइटमा कसरी फरक तथ्यांक देखियो त ? नेप्सको प्रवक्ता मुराहरी पराजुली भन्छन्, ‘यो प्राविधिक समस्या हो, सिस्टममा रिफ्रेस नभएर त्यस्तो देखिएको हुनसक्छ ।’ कारोबार बन्द हुने वित्तिकै र केहि मिनेटपछि भने तथ्यांक तलमाथि हुन्छ । अपडेट हुनलाई केही समय लाग्ने भएकाले यस्तो हुने गरेको उनले बताए ।
तर बुधबार कारोबार बन्द भएको ४० मिनेटसम्म पनि यस्तो देखिनु प्राविधिक समस्या भएको र पुरानो वेवसाइटलाई विस्वास गर्नु पर्ने उनले बताए ।
‘खासमा आधिकारिक भनेको नयाँ होइन पुरानै हो, यसरी तथ्यांक फरक परेको अवस्थामा पुरानोलाई आधार मान्नुपर्छ’ प्रवक्ता पराजुलीले भने । बुधबार आज सबै समूहका परिसूचक नकारात्मक रहे । यसमा पनि सबैभन्दा बढी गुमाउनेमा विकास बैंक समुहका लगानीकर्ता परे । जलविद्युत, जीवन बिमा र होटेल समुहमा समेत पहिरो गयो । सेन्सेटिभ इन्डेक्समा ७.३७ अंकको करेक्सन देखियो ।
बुधबार कुल २१८ कम्पनीको ९१ हजार ७५ वटा कारोबार भएको छ । कुल १ करोड ९४ लाख १७ हजार ५ सय ३२ कित्ता सेयर किनबेच भएको दिन बुधबार कारोबार रकम भने साढे ८ अर्बमा खुम्चिएको छ ।
बजार लगातार घटेको समयमा पनि कारोबार रकम १० अर्ब भन्दामाथि हुने गरेको थियो । आज ८ अर्ब ३६ करोड २६ लाख ७१ हजार ६ सय ८१ रुपैयाँ बराबरको कारोबार भएको छ ।
'''
label = 'Economy'

In [39]:
svc_clf.predict([text])[0]

'Economy'

In [ ]:
mnb_clf.predict([text])

In [ ]:
bnb_clf.predict([text])

In [ ]:
rf_clf.predict([text])